In [27]:
# import all the necessary libraries

from langchain.document_loaders.csv_loader import CSVLoader
import google.generativeai as genai
import google.ai.generativelanguage as glm
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
import os

In [28]:
# use google ai studio to get your api key
google_api_key = os.getenv('API_KEY')

In [29]:
# configure gemini pro
google_genai = ChatGoogleGenerativeAI(
    google_api_key=google_api_key,
    model='gemini-pro')

In [32]:
# embeddings to store vectorized texts
embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=google_api_key
)

In [33]:
# load the prompt files
loader = CSVLoader(file_path='prompt_and_response.csv', source_column='prompt')
data = loader.load()

In [34]:
# intialize the vector db
vector_db = FAISS.from_documents(documents=data, embedding=embeddings)

In [35]:
# get the retriver object from vector db
retriever = vector_db.as_retriever()
relavent_docs = retriever.get_relevant_documents('How about job support?')

In [36]:
# sample answer from prompt
relavent_docs

[Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
 Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: How can I get help if I have a doubt and need support?\nresponse: We have an active discord server where you can post your question and most of the time you will get an answer in a reasonable time frame.', metadata={'source': 'How can I get help if I have a doubt and need support?', 'row': 23}),
 Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13})]

In [25]:
# sample prompt template

prompt_template = """Given the following context and a question, generate an answer based on this context only.
    In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
    If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

    CONTEXT: {context}

    QUESTION: {question}"""